In [ ]:
import psycopg2
from sentence_transformers import SentenceTransformer
import pandas as pd  # Ensure this is installed if using DataFrame (df2)

# Connection parameters
host = "localhost"
user = "your_username"
password = "your_password"
port = "5432"
database = "database_name"  # Replace with your database name

try:
    # Step 1: Connect to PostgreSQL and setup pgvector
    connection = psycopg2.connect(
        host=host,
        user=user,
        password=password,
        port=port,
        database=database
    )
    connection.autocommit = True
    cursor = connection.cursor()

    # Enable pgvector extension
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("pgvector extension is ready.")

    # Create or modify the table for storing question embeddings
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS qa1_embeddings (
            id SERIAL PRIMARY KEY,
            question TEXT,
            answer TEXT,  -- New column for answers
            embedding vector(384)  -- 384 dimensions for all-MiniLM-L6-v2 model
        );
    """)
    print("Table 'qa1_embeddings' with 'answer' column is ready.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

# Step 2: Load Sentence-Transformers Model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def generate_embedding(text, model):
    return model.encode(text).tolist()  # Convert to list for PostgreSQL insertion


# Ensure your data has 'question' and 'answer' columns
data =df=pd.read_csv("question_answer.csv")


# Step 3: Insert Data into PostgreSQL
try:
    for index, row in df2.iterrows():  #\
        question = row["question"]
        answer = row["answer"]

        # Generate embedding for the question
        embedding = generate_embedding(question, model)

        # Insert into PostgreSQL
        cursor.execute("""
            INSERT INTO qa1_embeddings (question, answer, embedding)
            VALUES (%s, %s, %s);
        """, (question, answer, embedding))
        print(f"Inserted row {index + 1}: {question}")

except Exception as e:
    print("Error while processing data:", e)

# Close the connection
if 'connection' in locals() and connection:
    cursor.close()
    connection.close()
    print("\nPostgreSQL connection is closed.")
